<a href="https://colab.research.google.com/github/ajboulay/RAITE-TensorFlow/blob/master/RAITE_TensorFlow_Boulay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is 'live code' portion of the Reproducibility Tool. It appears in the left Hand window of the Reproducability Tool. On the right hand side is the Reproducability Checklist from NeurIPS. 

 A clear description of the mathematical setting, algorithm, and/or model. (Check)
A clear explanation of any assumptions.(Check)
An analysis of the complexity (time, space, sample size) of any algorithm.
For any theoretical claim, check if you include:
A clear statement of the claim. 
A complete proof of the claim.


# RAIT Reproducability Checklist 

For all models and algorithms presented, check if you include:
1. **A clear description of the mathematical setting, algorithm, and/or model.** 
*   The setting is that of a Tutorial and short introduction to Keras. Here we build a neural network that classifies images; we train this neural network and finally evaluate the accuracy of the model. The model is a TensorFlow Sequential model which is a liner feed forward network with four layers: an input layer, a ReLu layer, a Dropout layer and a Dense output layer. The mathematical setting retuns a vector of logits which is a vector of raw that a classification model generates, one for each class. These logits feed into a softmax function which converts these logits into probabilities for each class. 
2. **A clear explanation of any assumptions:** No assumptions are made, but a choice regarding **Responsible AI** is made by the authors. Specifically, they state that the softmax function can be put into the final output layer to improve **Interpretability**. **Interpretability** is an important factor in **Responsible AI**. However, the authors have chosen not to do this because it is 'impossible to provide an exact and numerically stable loss calculation for all models when using the softmax output.'
3. An analysis of the complexity (time, space, sample size) of any algorithm




In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0


Load MNIST dataset. 

# RAITE Reproducability Challenge Checklist 

*For all datasets used, check if you include:*
1. The relevant statistics, such as number of examples.   
2. A link to a downloadable version of the dataset or simulation environment. 
3. An explanation of any data that were excluded, and all pre-processing steps. 
4. The details of train / validation / test splits. 
5. For new data collected, a complete description of the data collection process, such as instructions to annotators and methods for quality control.









In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


# RAITE - Reproducability Checklist
1. This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. 
2. More info can be found at the (MNIST homepage)[http://yann.lecun.com/exdb/mnist/].

3. This dataset is a subset of a larger datset NIST (MNIST Homepage) [http://yann.lecun.com/exdb/mnist/]. The data have been size normalized and centered in a fixed-sized image. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image by computing the center of mass of the pixels, and translating the image so as to position this point at the center of the 28x28 field. The MNIST database was constructed from NIST's Special Database 3 and Special Database 1 which contain binary images of handwritten digits. NIST originally designated SD-3 as their training set and SD-1 as their test set. However, SD-3 is much cleaner and easier to recognize than SD-1. The reason for this can be found on the fact that SD-3 was collected among Census Bureau employees, while SD-1 was collected among high-school students. Drawing sensible conclusions from learning experiments requires that the result be independent of the choice of training set and test among the complete set of samples. Therefore it was necessary to build a new database by mixing NIST's datasets. 
4. The MNIST training set is composed of 30,000 patterns from SD-3 and 30,000 patterns from SD-1. Our test set was composed of 5,000 patterns from SD-3 and 5,000 patterns from SD-1. The 60,000 pattern training set contained examples from approximately 250 writers. We made sure that the sets of writers of the training set and test set were disjoint. SD-1 contains 58,527 digit images written by 500 different writers. In contrast to SD-3, where blocks of data from each writer appeared in sequence, the data in SD-1 is scrambled. Writer identities for SD-1 is available and we used this information to unscramble the writers. We then split SD-1 in two: characters written by the first 250 writers went into our new training set. The remaining 250 writers were placed in our test set. Thus we had two sets with nearly 30,000 examples each. The new training set was completed with enough examples from SD-3, starting at pattern # 0, to make a full set of 60,000 training patterns. Similarly, the new test set was completed with SD-3 examples starting at pattern # 35,000 to make a full set with 60,000 test patterns. Only a subset of 10,000 test images (5,000 from SD-1 and 5,000 from SD-3) is available on this site. The full 60,000 sample training set is available.
5. No new data was collected or added to this MNIST dataset. 

Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# RAITE - Reproducibility Checklist


For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [0]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.03730418, -0.49223322, -0.24446884,  0.67065024, -0.61294836,
         0.3236067 ,  0.24669674,  0.44222248,  0.28978026,  0.04779994]],
      dtype=float32)

The tf.nn.softmax function converts these logits to "probabilities" for each class:

In [0]:
tf.nn.softmax(predictions).numpy()

array([[0.08407824, 0.05334705, 0.06834602, 0.17066495, 0.04728077,
        0.12062164, 0.11169241, 0.1358124 , 0.11660968, 0.09154695]],
      dtype=float32)

Note: It is possible to bake this tf.nn.softmax in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.log(1/10) ~= 2.3.

In [0]:
loss_fn(y_train[:1], predictions).numpy()

2.1150966

In [0]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss: 

In [0]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2942 - accuracy: 0.9146
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1428 - accuracy: 0.9580
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1066 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0869 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0759 - accuracy: 0.9770


The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [0]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0670 - accuracy: 0.9803


[0.06704220920801163, 0.9803000092506409]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [0]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [0]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[9.30263866e-08, 1.34116940e-08, 1.99836518e-06, 3.25699148e-05,
        5.04651095e-12, 1.12387092e-07, 9.86155822e-15, 9.99963045e-01,
        3.12215676e-08, 2.16426633e-06],
       [4.03270217e-09, 6.87331485e-05, 9.99875546e-01, 4.11831970e-05,
        7.07910699e-14, 2.47920923e-07, 9.35004755e-06, 2.58046362e-11,
        5.06312472e-06, 1.07002945e-14],
       [3.21906498e-07, 9.99026179e-01, 7.58643073e-05, 8.88969589e-06,
        1.33073836e-05, 1.43751613e-05, 8.04785086e-05, 2.47585791e-04,
        5.32835256e-04, 1.63826030e-07],
       [9.99912620e-01, 2.06290851e-09, 2.15467899e-05, 1.94021467e-07,
        1.22967933e-07, 3.48462464e-07, 1.76327649e-05, 1.64222420e-05,
        8.06301500e-07, 3.02662556e-05],
       [1.20018276e-05, 5.22498851e-07, 1.81634168e-05, 3.63254102e-08,
        9.96430457e-01, 1.13101419e-06, 1.49250400e-05, 2.96129822e-03,
        7.23171843e-06, 5.54272381e-04]], dtype=float32)>